In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from datetime import datetime, timezone
import pytz
import glob




In [ ]:
simtel_dir="/Users/vdk/CTA/LST/4LSTmuons/"

simtel_files="simtel_run*_muon.simtel.gz"

for single_file in glob.glob(simtel_dir+simtel_files):
    print(single_file)

In [ ]:
int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

for k in range(1,10):
    filename = f'/Users/vdk/CTA/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    checker = 0
    for i,j in enumerate(event_iterator):
        for m in j.trigger.tels_with_trigger:
            checker += 1
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(j)
            image_processor(j)
            muon_processor(j)
            for m in j.muon.tel.keys():
                if not np.isnan(j.muon.tel[m].efficiency.optical_efficiency):
                    event = j
                    break
    break
    t2 = time.perf_counter(), time.process_time()
    print("checker = ", checker)   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
if event.muon.tel[1].efficiency.is_valid and not event.muon.tel[1].efficiency.parameters_at_limit:
    print('hi')

In [ ]:
event.muon.tel[1].efficiency.is_valid = True

In [ ]:
def test_func(param):
    if param:
        print('hi')

In [ ]:
test_func(False)

In [ ]:
filename = f'/Users/vdk/Software/ctasoft/calibpipe/calibpipe/tests/data/throughput/simtel_run501_muon_telescope_transmission_0.8.simtel.gz'
print(filename)
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
event = next(event_iterator)

In [ ]:
event = next(event_iterator)
timestamp = event.trigger.tel[3].time.utc.to_datetime(timezone.utc)
timestamp.tzinfo.utcoffset()

In [ ]:
timestamp = timestamp.utc.to_datetime(timezone.utc)
event.trigger.tel[3].time

In [ ]:
dt_utc = timestamp.astimezone(pytz.utc)
unix_timestamp = float(dt_utc.timestamp())
type(unix_timestamp)

In [ ]:
from astropy.table import Table
import pytz

# Replace 'Your_Timezone' with your timezone, e.g., 'America/New_York'
your_timezone = pytz.timezone('')

# Get the current time in your timezone
now_in_your_timezone = datetime.now(your_timezone)

# Get the timedelta of your timezone
timezone_timedelta = now_in_your_timezone.utcoffset()

In [ ]:
all_muon_parameters = ["obs_id", "timestamp", "optical_efficiency"]
table = Table(names=all_muon_parameters)
table

In [ ]:
timestamp.isoformat()

In [ ]:
date_string = "21 June, 2018"

print("date_string =", date_string)
print("type of date_string =", type(date_string))

date_object = datetime.strptime(date_string, "%d %B, %Y")

print("date_object =", date_object)
print("type of date_object =", type(date_object))

In [ ]:
testtime = event.trigger.tel[1].time.to_value("mjd", "str")

In [ ]:
date_string = testtime

print("date_string =", date_string)
print("type of date_string =", type(date_string))

date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

print("date_object =", date_object)
print("type of date_object =", type(date_object))